In [1]:
import json
import random
import torch

In [2]:
import pandas as pd

dataset = pd.read_csv("data_full_v1.csv")

In [3]:
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)

dataset['prompt'] = dataset.apply(lambda row: (row['prompt']).removeprefix('Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: '), axis=1)


In [4]:
dataset['prompt'] = dataset['prompt'].str.lower()

In [5]:
dataset

,id,prompt,label,ground_truth
0,4,husband has a direct effect on wife and alarm ...,yes,1
1,7,husband has a direct effect on wife and alarm ...,no,0
2,8,husband has a direct effect on wife and alarm ...,yes,1
3,15,husband has a direct effect on wife and alarm ...,no,0
4,21,husband has a direct effect on wife and alarm ...,no,0
...,...,...,...,...
10107,31012,zuph has a direct effect on glimx. jyka has a ...,yes,1
10108,31014,zuph has a direct effect on glimx. jyka has a ...,yes,1
10109,31015,zuph has a direct effect on glimx. jyka has a ...,no,0
10110,31016,zuph has a direct effect on glimx. jyka has a ...,no,0


In [6]:
dataset = json.loads(dataset.to_json(orient ='records'))

# Shuffle the dataset
random.shuffle(dataset)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
validation_data = dataset[train_size:]

In [7]:
from torch.utils.data import Dataset

class CladderDataset(Dataset):
    def __init__(self, cladder_data, tokenizer):
        self.cladder_data = cladder_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.cladder_data)

    def __getitem__(self, idx):
        data = self.cladder_data[idx]
        # print(data)
        prompt = data['prompt']
        label = data['ground_truth']

        # print(prompt)
        # print(label)

        # Tokenize
        encoded_input = self.tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').to("cuda")
        # print(encoded_input)
        encoded_input['labels'] = torch.tensor(label).to("cuda")

        return encoded_input


In [8]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

# Load tokenizer and model
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)

In [10]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [11]:
# # Prepare the DataLoader
# cladder_dataset = CladderDataset(train_data, tokenizer)
# dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

# # Optimizer and Loss Function
# optimizer = optim.Adam(model.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import RobertaTokenizer as tkz
from transformers import RobertaForSequenceClassification as mpc

# Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

# run = 0
# for run in range(1):

for run in range(3):
  model_base = 'roberta-base'
  tokenizer = tkz.from_pretrained(model_base)
  model = mpc.from_pretrained(model_base, num_labels = 2)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  # Prepare the DataLoader
  cladder_dataset = CladderDataset(train_data, tokenizer)
  dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

  # Optimizer and Loss Function
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  loss_fn = torch.nn.CrossEntropyLoss()



  # Training Loop
  model.train()

  #change to 3

  for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  torch.save(model.state_dict(), model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.6248730421066284
Epoch: 0, Loss: 0.6802446842193604
Epoch: 0, Loss: 0.6617556810379028
Epoch: 0, Loss: 0.5687083601951599
Epoch: 0, Loss: 0.6413174271583557
Epoch: 0, Loss: 0.5820117592811584
Epoch: 0, Loss: 0.6822767853736877
Epoch: 0, Loss: 0.9149326682090759
Epoch: 0, Loss: 0.8771120309829712
Epoch: 0, Loss: 0.6081929802894592
Epoch: 0, Loss: 0.6719855666160583
Epoch: 0, Loss: 0.6948096752166748
Epoch: 0, Loss: 0.501382052898407
Epoch: 0, Loss: 0.7786209583282471
Epoch: 0, Loss: 0.8592779040336609
Epoch: 0, Loss: 0.6322096586227417
Epoch: 0, Loss: 0.7694776058197021
Epoch: 0, Loss: 0.8528417944908142
Epoch: 0, Loss: 0.8221146464347839
Epoch: 0, Loss: 0.7294208407402039
Epoch: 0, Loss: 0.6607663631439209
Epoch: 0, Loss: 0.7265591025352478
Epoch: 0, Loss: 0.6323931813240051
Epoch: 0, Loss: 0.7542137503623962
Epoch: 0, Loss: 0.6137346625328064
Epoch: 0, Loss: 0.7750809192657471
Epoch: 0, Loss: 0.7183672189712524
Epoch: 0, Loss: 0.7845175266265869
Epoch: 0, Loss: 0.671

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7098004817962646
Epoch: 0, Loss: 0.6931498646736145
Epoch: 0, Loss: 0.7249874472618103
Epoch: 0, Loss: 0.7126112580299377
Epoch: 0, Loss: 0.7241119146347046
Epoch: 0, Loss: 0.6961692571640015
Epoch: 0, Loss: 0.6962407827377319
Epoch: 0, Loss: 0.7020931243896484
Epoch: 0, Loss: 0.6548841595649719
Epoch: 0, Loss: 0.7036685347557068
Epoch: 0, Loss: 0.6311161518096924
Epoch: 0, Loss: 0.6599619388580322
Epoch: 0, Loss: 0.6228980422019958
Epoch: 0, Loss: 0.705996036529541
Epoch: 0, Loss: 0.6566805839538574
Epoch: 0, Loss: 0.7278019189834595
Epoch: 0, Loss: 0.7397823929786682
Epoch: 0, Loss: 0.7684949040412903
Epoch: 0, Loss: 0.7156769037246704
Epoch: 0, Loss: 0.7624168395996094
Epoch: 0, Loss: 0.708034873008728
Epoch: 0, Loss: 0.7122683525085449
Epoch: 0, Loss: 0.802595853805542
Epoch: 0, Loss: 0.6255388259887695
Epoch: 0, Loss: 0.7337443232536316
Epoch: 0, Loss: 0.6515659689903259
Epoch: 0, Loss: 0.7470853328704834
Epoch: 0, Loss: 0.7927641868591309
Epoch: 0, Loss: 0.64269

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7044334411621094
Epoch: 0, Loss: 0.7027466893196106
Epoch: 0, Loss: 0.7023485898971558
Epoch: 0, Loss: 0.6610421538352966
Epoch: 0, Loss: 0.6989205479621887
Epoch: 0, Loss: 0.6828115582466125
Epoch: 0, Loss: 0.6985041499137878
Epoch: 0, Loss: 0.6878169775009155
Epoch: 0, Loss: 0.6899222731590271
Epoch: 0, Loss: 0.6810617446899414
Epoch: 0, Loss: 0.6893414258956909
Epoch: 0, Loss: 0.700957179069519
Epoch: 0, Loss: 0.688616931438446
Epoch: 0, Loss: 0.678216814994812
Epoch: 0, Loss: 0.6910706162452698
Epoch: 0, Loss: 0.6451815366744995
Epoch: 0, Loss: 0.6870739459991455
Epoch: 0, Loss: 0.7379388809204102
Epoch: 0, Loss: 0.6995618939399719
Epoch: 0, Loss: 0.7271543145179749
Epoch: 0, Loss: 0.6593211889266968
Epoch: 0, Loss: 0.701301634311676
Epoch: 0, Loss: 0.7099677324295044
Epoch: 0, Loss: 0.6985318660736084
Epoch: 0, Loss: 0.7165460586547852
Epoch: 0, Loss: 0.7078529000282288
Epoch: 0, Loss: 0.722893238067627
Epoch: 0, Loss: 0.6917884945869446
Epoch: 0, Loss: 0.7033573

In [ ]:
# # Training Loop
# model.train()

# for epoch in range(3):  # Number of epochs
#   for batch in dataloader:
#       optimizer.zero_grad()

#       input_ids = batch['input_ids'].squeeze(1)
#       attention_mask = batch['attention_mask'].squeeze(1)
#       labels = batch['labels']

#       outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#       loss = outputs.loss

#       loss.backward()
#       optimizer.step()

#       print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.7476723194122314
Epoch: 0, Loss: 0.7382199764251709
Epoch: 0, Loss: 0.7543107271194458
Epoch: 0, Loss: 0.6875097751617432
Epoch: 0, Loss: 0.7356888055801392
Epoch: 0, Loss: 0.627315878868103
Epoch: 0, Loss: 0.6803908944129944
Epoch: 0, Loss: 0.7097512483596802
Epoch: 0, Loss: 0.6597291827201843
Epoch: 0, Loss: 0.6984984874725342
Epoch: 0, Loss: 0.6785147190093994
Epoch: 0, Loss: 0.6503101587295532
Epoch: 0, Loss: 0.7109977006912231
Epoch: 0, Loss: 0.6658034324645996
Epoch: 0, Loss: 0.7305421829223633
Epoch: 0, Loss: 0.7359177470207214
Epoch: 0, Loss: 0.7260733246803284
Epoch: 0, Loss: 0.7163431644439697
Epoch: 0, Loss: 0.712849497795105
Epoch: 0, Loss: 0.6823022365570068
Epoch: 0, Loss: 0.7044941782951355
Epoch: 0, Loss: 0.7307822704315186
Epoch: 0, Loss: 0.7365822792053223
Epoch: 0, Loss: 0.7116348147392273
Epoch: 0, Loss: 0.7333304286003113
Epoch: 0, Loss: 0.6940729022026062
Epoch: 0, Loss: 0.7345658540725708
Epoch: 0, Loss: 0.6587915420532227
Epoch: 0, Loss: 0.6933

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

overall_accuracies, accuracies, f1s, precisions, recalls = [], [], [], [], []

# for run in range(1):
for run in range(3):
  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  print(model_path)

  model = mpc.from_pretrained(model_base, num_labels = 2)
  model.load_state_dict(torch.load(model_path))

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  tokenizer = tkz.from_pretrained(model_base)
  validation_dataset = CladderDataset(validation_data, tokenizer)
  validation_dataloader = DataLoader(validation_dataset, batch_size=8)




  model.eval()
  correct_predictions = 0
  total_predictions = 0

  accuracy, f1, precision, recall = [], [], [], []
  batches = 0

  def compute_metrics(y_true, y_pred):
      accuracy = accuracy_score(y_true, y_pred)
      # print(y_true)
      # print(y_pred)
      f1 = f1_score(y_true, y_pred)

      precision = precision_score(y_true, y_pred)
      # print(precision)
      recall = recall_score(y_true, y_pred)
      return [accuracy, f1, precision, recall]

  with torch.no_grad():
      for batch in validation_dataloader:
          input_ids = batch['input_ids'].squeeze(1)
          attention_mask = batch['attention_mask'].squeeze(1)
          labels = batch['labels'].cpu()  # ground truth
          # print(labels)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          # print(outputs)
          # print(outputs.logits)
          predictions = torch.argmax(outputs.logits, dim=1).cpu()  # predictions
          # print(predictions)

          calculated_metrics = compute_metrics(labels, predictions)
          accuracy.append(calculated_metrics[0])
          f1.append(calculated_metrics[1])
          precision.append(calculated_metrics[2])
          recall.append(calculated_metrics[3])
          batches += 1

          correct_predictions += (predictions == labels).sum().item()
          total_predictions += labels.size(0)

  overall_accuracy = correct_predictions / total_predictions
  print(f"Overall Accuracy: {overall_accuracy:.4f}")
  overall_accuracies.append(overall_accuracy)

  accuracy_var = sum(accuracy)/batches
  print(f"Accuracy: {sum(accuracy)/batches:.4f}")
  print(accuracy_var)
  accuracies.append(accuracy_var)

  f1_var = sum(f1)/batches
  print(f"F1: {sum(f1)/batches:.4f}")
  print(f1_var)
  f1s.append(f1_var)

  precision_var = sum(precision)/batches
  print(f"Precision: {sum(precision)/batches:.4f}")
  print(precision_var)
  precisions.append(precision_var)

  recall_var = sum(recall)/batches
  print(f"Recall: {sum(recall)/batches:.4f}")
  print(recall_var)
  recalls.append(precision_var)

  print(f'Number of batches: {batches}')

cladder_roberta-base_0.pt


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Overall Accuracy: 0.5036
Accuracy: 0.5036
0.5035601265822784
F1: 0.6527
0.6527269495623926
Precision: 0.5036
0.5035601265822784
Recall: 1.0000
1.0
Number of batches: 316
cladder_roberta-base_1.pt


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# validation_dataset = CladderDataset(validation_data, tokenizer)
# validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [ ]:
# model.eval()
# correct_predictions = 0
# total_predictions = 0

# with torch.no_grad():
#     for batch in validation_dataloader:
#         input_ids = batch['input_ids'].squeeze(1)
#         attention_mask = batch['attention_mask'].squeeze(1)
#         labels = batch['labels']

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         predictions = torch.argmax(outputs.logits, dim=1)
#         correct_predictions += (predictions == labels).sum().item()
#         print(f"predictions: {predictions}, labels: {labels}")
#         total_predictions += labels.size(0)

# print(f"total_predictions: {total_predictions}")

# overall_accuracy = correct_predictions / total_predictions
# print(f"Overall Accuracy: {overall_accuracy:.4f}")


predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([1, 0, 1, 0, 0, 1, 0, 0], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([0, 0, 1, 0, 1, 1, 0, 1], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([1, 1, 0, 0, 1, 1, 1, 0], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([0, 0, 1, 0, 1, 0, 1, 1], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([1, 1, 1, 0, 0, 0, 0, 1], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([1, 0, 1, 1, 0, 0, 1, 1], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([1, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')
predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), labels: tensor([0, 0, 1, 1, 1, 0, 1, 1], device='cuda:0')
predictions: tensor([1, 